Install Dependencies:


1.   Java 8
2.   Apache Spark with hadoop and
3.   Findspark (used to locate the spark in the system)

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz

Set Environment Variables:

In [ ]:
import os
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
!pip install findspark
import findspark
findspark.init()

In [ ]:
!pip3 install pyspark==3.0.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.7/204.7 MB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 1.8 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.0.0-py2.py3-none-any.whl size=205044159 sha256=6a9ca4dbf9ad916c286bf0a4ad707b0bd0e3ca995c8ea4526d8b832dd4d9d309
  Stored in directory: /root/.cache/pip/wheels/b1/bb/8b/ca24d3f756f2ed967225b0871898869db676eb5846df5adc56
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [ ]:
from pyspark import SparkContext, SparkConf
import pyspark.sql as sql
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import udf, col, max, sum, countDistinct

In [ ]:
spark = SparkSession \
    .builder \
    .appName("L1_interactive_bike_analysis") \
    .getOrCreate()

In [ ]:
import os
data_path = os.path.join(os.curdir, "data")
trips_path = os.path.join("trips.csv")
stations_path = os.path.join("stations.csv")

In [ ]:
trips = spark.read \
        .option("header", True) \
        .option("inferSchema", True) \
        .option("DateTimeFormat", 'M/d/y H:m') \
        .csv(trips_path)

stat = spark.read \
        .option("header", True) \
        .option("inferSchema", True) \
        .option("DateTimeFormat", 'M/d/y H:m') \
        .csv(stations_path)

<a id='Задачи для данных велопарковок Сан-Франциско (trips.csv, stations.csv):'></a>
## Задачи для данных велопарковок Сан-Франциско (trips.csv, stations.csv):
><li>Найти велосипед с максимальным временем пробега.
><li>Найти наибольшее геодезическое расстояние между станциями.
><li>Найти путь велосипеда с максимальным временем пробега через станции.
><li>Найти количество велосипедов в системе.
><li>Найти пользователей потративших на поездки более 3 часов.

1. Найти велосипед с максимальным временем пробега

In [ ]:
#Группировка велосипедов по id; подсчет времени пробега для каждого
trip_dur = trips.groupBy("bike_id").agg(sum(col("duration")).alias("duration"))
#Находим велосипед с максимальным пробегом
trips_dur = trip_dur.orderBy(col("duration").desc()).first()
bike_id = trips_dur["bike_id"]
dur = trips_dur["duration"]
print(f"Велосипед {bike_id} с макс временем {dur}")

Велосипед 168 с макс временем 613841


2. Найти наибольшее геодезическое расстояние между станциями

In [ ]:
from math import sqrt, radians, sin, cos, atan2

def dist(l1, long1, l2, long2):
    R = 6373
    l1 = radians(l1)
    l2 = radians(l2)

    long1 = radians(long1)
    long2 = radians(long2)

    dist = R * (2 * atan2(sqrt(sin((l2 - l1) / 2)**2 + cos(l1) * cos(l2) * sin((long2 - long1) / 2)**2), sqrt(1 - (sin((l2 - l1) / 2)**2 + cos(l1) * cos(l2) * sin((long2 - long1) / 2)**2))))

    return dist


geod_dist = udf(dist, DoubleType())
#Все пары станций
station = stat.alias("station1").crossJoin(stat.alias("station2"))
#Расстояние для каждой пары
station_distance = station.withColumn("geodesic_distance", geod_dist(col("station1.lat"), col("station1.long"), col("station2.lat"), col("station2.long")))

dist = station_distance.selectExpr("max(geodesic_distance) as max_distance").collect()[0]["max_distance"]
print(f"Наибольшее расстояние: {dist}")

Наибольшее расстояние: 69.9428256877473


3. Найти путь велосипеда с максимальным временем пробега через станции

In [ ]:
#Поиск наиболее длительной поездки через сортировку
dur = trips.select("start_station_name", "end_station_name", "duration").orderBy(col("duration").desc()).first()

one = dur["start_station_name"] #Начальная станция
two = dur["end_station_name"] #Конечная станция
time = dur["duration"] #Время поездки

print(f"Самая длинная поездка: из \"{one}\" в \"{two}\" занимает {time}")

Самая длинная поездка: из "California Ave Caltrain Station" в "Palo Alto Caltrain Station" занимает 597517


4. Найти количество велосипедов в системе

In [ ]:
count = trips.agg(countDistinct("bike_id").alias("bike_count")).collect()[0]["bike_count"]

print(f"Количество велосипедов: {count}")

Количество велосипедов: 622


5. Найти пользователей потративших на поездки более 3 часов

In [ ]:
#Группируем по id и считаем общее время
user = trips.groupBy("bike_id").sum("duration").withColumnRenamed("sum(duration)", "time")

user.filter("time>10800").show()

+-------+------+
|bike_id|  time|
+-------+------+
|    471| 83444|
|    496|103096|
|    148| 99266|
|    540|158904|
|    392|121202|
|    623|207944|
|    516| 72546|
|    580| 90171|
|    137| 44684|
|    458|112291|
|    588| 85735|
|    255| 52279|
|     53| 18442|
|    481| 70738|
|    472| 97185|
|    322| 80487|
|    513|133817|
|    613|266889|
|    633|138351|
|    321| 88916|
+-------+------+
only showing top 20 rows

